<a href="https://colab.research.google.com/github/Kavinkumar-E/Machine-learning--2024dc04106/blob/main/train_models_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q scikit-learn xgboost pandas numpy

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving heart.csv to heart.csv


In [ ]:
import pandas as pd
import numpy as np
import logging

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    precision_score,
    recall_score,
    f1_score,
    matthews_corrcoef
)

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


# ---------------------------------------------------
# LOGGING CONFIGURATION
# ---------------------------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s"
)

logger = logging.getLogger()


def evaluate(model, X_test, y_test):
    logger.info("Evaluating model performance")

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    metrics = {
        "Accuracy": accuracy_score(y_test, y_pred),
        "AUC": roc_auc_score(y_test, y_prob),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1": f1_score(y_test, y_pred),
        "MCC": matthews_corrcoef(y_test, y_pred)
    }

    logger.info("Evaluation completed")
    return metrics


def train_all_models(df):
    logger.info("Dataset uploaded successfully")
    logger.info(f"Dataset shape: {df.shape}")

    logger.info("Separating features and target column")
    X = df.drop("HeartDisease", axis=1)
    y = df["HeartDisease"]

    logger.info("Applying one-hot encoding")
    X = pd.get_dummies(X, drop_first=True)

    logger.info("Splitting dataset into train and test sets")
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    logger.info("Applying standard scaling")
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    models = {
        "Logistic Regression": LogisticRegression(max_iter=1000),
        "Decision Tree": DecisionTreeClassifier(random_state=42),
        "KNN": KNeighborsClassifier(),
        "Naive Bayes": GaussianNB(),
        "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
        "XGBoost": XGBClassifier(
            use_label_encoder=False,
            eval_metric="logloss",
            random_state=42
        )
    }

    results = {}

    for name, model in models.items():
        logger.info(f"Training started for {name}")
        model.fit(X_train, y_train)

        logger.info(f"Training completed for {name}")
        results[name] = evaluate(model, X_test, y_test)

        logger.info(f"Metrics stored for {name}")

    logger.info("All models executed successfully")
    return results


In [ ]:
# Load dataset
df = pd.read_csv("heart.csv")

# Train and evaluate models
results = train_all_models(df)

# Display results
results_df = pd.DataFrame(results).T
results_df


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:01:03] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


,Accuracy,AUC,Precision,Recall,F1,MCC
Logistic Regression,0.853261,0.926690,0.900000,0.841121,0.869565,0.704402
Decision Tree,0.826087,0.823158,0.857143,0.841121,0.849057,0.644131
KNN,0.853261,0.923656,0.900000,0.841121,0.869565,0.704402
Naive Bayes,0.858696,0.930210,0.909091,0.841121,0.873786,0.716625
Random Forest,0.875000,0.935186,0.896226,0.887850,0.892019,0.743686
XGBoost,0.869565,0.921107,0.895238,0.878505,0.886792,0.733160
